In [ ]:

import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from pytrends.request import TrendReq
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools.convert_to_openai import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from textblob import TextBlob 
import wbdata
import yfinance as yf
from ta import add_all_ta_features
from ta.utils import dropna

@tool
def technical_analysis(ticker, period):
    """Perform technical analysis for a stock"""
    try:
        # Get historical stock data
        stock_data = yf.download(ticker, period=period)

        # Add technical analysis features
        stock_data = dropna(stock_data)
        stock_data = add_all_ta_features(
            stock_data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True
        )

        # Extract relevant technical indicators
        indicators = {
            "rsi": stock_data["momentum_rsi"],
            "macd": stock_data["trend_macd_diff"],
            "ma50": stock_data["trend_sma_fast"],
            "ma200": stock_data["trend_sma_slow"],
            "bb_bbm": stock_data["volatility_bbm"],
            "bb_bbh": stock_data["volatility_bbh"],
            "bb_bbl": stock_data["volatility_bbl"],
            "stoch": stock_data["momentum_stoch"],
            "atr": stock_data["volatility_atr"],
        }

        # Analyze indicators and provide buy/sell signals
        signals = {}
        for indicator, values in indicators.items():
            if values.iloc[-1] > values.iloc[-2]:
                signals[indicator] = "Strong Buy"
            elif values.iloc[-1] < values.iloc[-2]:
                signals[indicator] = "Strong Sell"
            elif values.iloc[-1] > values.iloc[-3]:
                signals[indicator] = "Weak Buy"
            elif values.iloc[-1] < values.iloc[-3]:
                signals[indicator] = "Weak Sell"
            else:
                signals[indicator] = "Neutral"

        return signals
    except Exception as e:
        return f"Failed to perform technical analysis for {ticker}. Error: {str(e)}"

@tool
def get_current_stock_price(ticker):
    """Method to get current stock price"""

    ticker_data = yf.Ticker(ticker)
    recent = ticker_data.history(period="1d")
    return {"price": recent.iloc[0]["Close"], "currency": ticker_data.info["currency"]}


@tool
def get_top_company_news(ticker, num_articles=10):
    """get company headlines- can be used to see top news concerning company and for further qualitative evaluation on price movements """
        # Define the URL for the Google News API
    url = f"https://news.google.com/rss/search?q={ticker}&hl=en-US&gl=US&ceid=US:en"

    # Send an HTTP GET request to the API
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the XML response
        import xml.etree.ElementTree as ET
        root = ET.fromstring(response.text)

        # Initialize a list to store news articles
        articles = []

        # Iterate through the XML to extract article titles and links
        for item in root.findall(".//item")[:num_articles]:
            title = item.find("title").text
            link = item.find("link").text
            articles.append({"title": title, "link": link})

        return articles
    else:
        return (
            f"Failed to retrieve news for {ticker}. Status code: {response.status_code}")

@tool
def get_historical_stock_data(ticker, start_date, end_date):
    """Get historical stock data"""
    ticker_data = yf.Ticker(ticker)
    historical_data = ticker_data.history(start=start_date, end=end_date)
    return historical_data.to_dict(orient='records')

@tool
def sentiment_analysis(text):
    """Perform sentiment analysis on text"""
    analysis = TextBlob(text)
    # Classify the sentiment as positive, negative, or neutral
    if analysis.sentiment.polarity > 0:
        return "Positive sentiment"
    elif analysis.sentiment.polarity < 0:
        return "Negative sentiment"
    else:
        return "Neutral sentiment"


@tool
def get_economic_indicators(country_code):
    """Get key economic indicators for a specific country"""
    indicators = {"NY.GDP.MKTP.CD": "GDP", "SL.UEM.TOTL.ZS": "Unemployment Rate", "FP.CPI.TOTL.ZG": "Inflation Rate"}

    try:
        data_date = datetime.datetime.now().year
        data = wbdata.get_dataframe(indicators, country=country_code, data_date=data_date, convert_date=False)

        if data.empty:
            return f"No data available for country code {country_code}"

        # Extract the latest available economic indicators
        latest_data = data.iloc[-1].to_dict()

        # Format the response
        formatted_response = {indicators[indicator]: f"{value:.2f}%" for indicator, value in latest_data.items()}
        return formatted_response
    except Exception as e:
        return f"Failed to fetch economic indicators for {country_code}. Error: {str(e)}"

@tool
def description_of_company(ticker):
    """get descriptions on company(summary,sector,country and industry)"""
    try:
        # Get financial statements
        company = yf.Ticker(ticker)
        return pd.DataFrame(company.info)[[ 'city', 'state', 'country',
       'industry', 'industryKey', 'industryDisp','sector', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', ]].iloc[0].to_dict()

    except Exception as e:
        return f"Failed to perform fundamental analysis for {ticker}. Error: {str(e)}"


@tool
def fundamental_analysis(ticker):
    """Perform fundamental analysis for a company"""
    try:
        # Get financial statements
        company = yf.Ticker(ticker)
        return pd.DataFrame(company.info)[[  'bookValue', 'priceToBook', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'mostRecentQuarter', 'earningsQuarterlyGrowth', 'netIncomeToCommon', 'trailingEps', 'forwardEps', 'pegRatio', 'totalCashPerShare', 'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 'returnOnAssets', 'returnOnEquity', 'freeCashflow', 'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'grossMargins', 'ebitdaMargins', 'operatingMargins', 'financialCurrency', 'trailingPegRatio']].iloc[0].to_dict()
        return company.info
    except Exception as e:
        return f"Failed to perform fundamental analysis for {ticker}. Error: {str(e)}"



@tool
def get_google_trends(keyword, timeframe='today 1-y', geo='US'):
    """
    Parameter 1. keyword: stockname,
    2. timeframe: eg. 'today 1-y'
    3. geo: eg. 'US' 
    """
    pytrends = TrendReq(hl='en-US', tz=360)
    
    # Build payload
    pytrends.build_payload([keyword], cat=0, timeframe=timeframe, geo=geo, gprop='')

    # Get interest over time
    interest_over_time_df = pytrends.interest_over_time()

    return interest_over_time_df

search = TavilySearchResults()

llm = ChatOpenAI(model="gpt-4", temperature=0.8)
tools = [
    get_top_company_news,
    get_current_stock_price,
    search,
    GoogleFinanceQueryRun(api_wrapper=GoogleFinanceAPIWrapper()),
    fundamental_analysis,
    get_economic_indicators,
    technical_analysis,
    description_of_company
    
]
chat_history = []
MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a powerful and versatile financial robo-advisor designed to assist users with a wide range of financial inquiries. Consider using the different tools based on different circumstances. Approach Each User's Question Step by Step.n\n\ 
           When asked about the performance of sector, query for top news related to sector and evaluate the content
           """
#             """You are a powerful and versatile financial robo-advisor designed to assist users with a wide range of financial inquiries. Consider using the different tools based on different circumstances.n\n\
#                 If asked to give recommendations - follow this thought process closely, using available tools (including online search if insufficient information available)
#                 '''1. Check for Over-exposure in a Particular Sector/Constituent:
# Assess the allocation of the portfolio to identify potential over-exposure in specific sectors or constituents.
# IF OVEREXPOSURE: 
# 2. Evaluate Risk Factors:
# Integrate a comprehensive risk assessment, considering factors such as volatility, historical drawdowns, and correlation risks.
# 3. Assess Trend (Downward Trending) & Fundamentals:
# Determine the overall trend (past 6 months) of over-exposed sectors or constituents by analyzing historical performance and recent market trends
# 4. Suggest Removal of Constituent:
# Recommend the removal of identified volatile or poor performing constituents to reduce overall portfolio risk 
# ELSE: 
# 5. Identify Untapped Sectors that are Promising:
# Explore untapped sectors with promising potential for diversification and growth.
# 6. Gather Potential Additional Constituents/Sector:
# Identify sectors or individual assets with low correlation to the removed constituent or sector.
# Propose potential additions to enhance diversification and mitigate correlation risks.
# 7. Evaluate Each Constituent's Technicals & Fundamentals:
# Conduct a thorough analysis of technical indicators and fundamental metrics for each constituent.
# Consider technical aspects such as Moving Averages, RSI, MACD, and fundamental aspects like EPS, P/E ratio, and Debt to Equity ratio.
# 8. Re-weight Constituents:
# Propose a strategic re-weighting plan for the remaining constituents, aiming for a more balanced and optimized allocation.
# '''
#            .          
#            """,
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])



agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [ ]:
inputval = f"""summarise google's recent performance"""

while inputval != 'thanks':
    result = agent_executor.invoke({"input": inputval, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=inputval),
            AIMessage(content=result["output"]),
        ]
    )
    inputval = input('what would you like to ask')




> Entering new AgentExecutor chain...

Invoking: `get_top_company_news` with `{'ticker': 'GOOGL', 'num_articles': 5}`


[{'title': 'Alphabet (GOOGL) Enhances Google Maps With New Feature - Yahoo Finance', 'link': 'https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9hbHBoYWJldC1nb29nbC1lbmhhbmNlcy1nb29nbGUtbWFwcy0xNjQ2MDAyNDcuaHRtbNIBAA?oc=5'}, {'title': "Here's Why It May Be Time To Sell Shares Of Google Stock - Investor's Business Daily", 'link': 'https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vcmVzZWFyY2gvZ29vZ2xlLXN0b2NrLXNlbGwtc2lnbmFscy1vcGVuYWktbWljcm9zb2Z0L9IBAA?oc=5'}, {'title': 'Alphabet (GOOGL) Stock Moves -0.51%: What You Should Know - Yahoo Finance', 'link': 'https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9hbHBoYWJldC1nb29nbC1zdG9jay1tb3Zlcy0wLTIyNDUxNzU4OC5odG1s0gEA?oc=5'}, {'title': "Google (NASDAQ:GOOGL) Makes Changes to Comply with EU's Digital Markets Act - TipRanks.com -

KeyboardInterrupt: 

In [ ]:
result["output"]

"The current stock price of Google (Alphabet Inc.) is $131.33 USD. \n\nIn terms of recent news:\n\n1. Alphabet is enhancing Google Maps with a new feature. [Read more](https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy9hbHBoYWJldC1nb29nbC1lbmhhbmNlcy1nb29nbGUtbWFwcy0xNjQ2MDAyNDcuaHRtbNIBAA?oc=5)\n2. There's an opinion piece suggesting it might be time to sell Google stock. [Read more](https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vd3d3LmludmVzdG9ycy5jb20vcmVzZWFyY2gvZ29vZ2xlLXN0b2NrLXNlbGwtc2lnbmFscy1vcGVuYWktbWljcm9zb2Z0L9IBAA?oc=5)\n3. Google has made changes to comply with the EU’s Digital Markets Act. [Read more](https://news.google.com/rss/articles/CBMiZWh0dHBzOi8vd3d3LnRpcHJhbmtzLmNvbS9uZXdzL2dvb2dsZS1uYXNkYXFnb29nbC1tYWtlcy1jaGFuZ2VzLXRvLWNvbXBseS13aXRoLWV1cy1kaWdpdGFsLW1hcmtldHMtYWN00gEA?oc=5)\n4. Alphabet has boosted its Android Auto application with a new AI feature. [Read more](https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vZmluYW